# Rekurencyjne Sieci Neuronowe (RNN)

### Importy i Utilsy  (odpalić i schować )

In [1]:
# imports 
import torch
import os
import unicodedata
import string
import numpy as np
from typing import Tuple, Optional, List

from torch.nn.functional import cross_entropy

import matplotlib.pyplot as plt 
from sklearn.metrics import f1_score

from torch.utils.data import Dataset, DataLoader

all_letters = string.ascii_letters
n_letters = len(all_letters)


class ListDataset(Dataset):
    
    def __init__(self, data, targets):
        
        self.data = data
        self.targets = targets
        
    def __getitem__(self, ind):
        
        return self.data[ind], self.targets[ind]
    
    def __len__(self):
        return len(self.targets)

    
def unicode_to__ascii(s: str) -> str:
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'
                                                                 and c in all_letters)
                   

def read_lines(filename: str) -> List[str]:
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicode_to__ascii(line) for line in lines]


def letter_to_index(letter: str) -> int:
    return all_letters.find(letter)


def line_to_tensor(line: str) -> torch.Tensor:
    tensor = torch.zeros(len(line), n_letters)
    for i, letter in enumerate(line):
        tensor[i][letter_to_index(letter)] = 1
    return tensor

## Dane sekwencyjne

Modele, którymi zajmowaliśmy się wcześniej zakładały konkretny kształt danych. Dla przykładu klasyczna sieć neuronowa fully-connected dla MNISTa zakładała, że na wejściu dostanie wektory rozmiaru 784 - dla wektorów o innej wymiarowości i innych obiektów model zwyczajnie nie będzie działać.

Takie założenie bywa szczególnie niewygodne przy pracy z niektórymi typami danych, takimi jak:
* językiem naturalny (słowa czy zdania mają zadanej z góry liczby znaków)
* szeregi czasowe (dane giełdowe ciągną się właściwie w nieskończoność) 
* dźwięk (nagrania mogą być krótsze lub dłuższe).

Do rozwiązania tego problemu służą rekuencyjne sieci neuronowe (*recurrent neural networks, RNNs*), które zapamiętują swój stan z poprzedniej iteracji.

### Ładowanie danych
Poniższe dwie komórki ściągają dataset nazwisk z 18 różnych narodowości. Każda litera w danym nazwisku jest zamieniana na jej indeks z alfabetu w postaci kodowania "one-hot". Inaczej mówiąc, każde nazwisko jest binarną macierzą rozmiaru `len(name)` $\times$ `n_letters`. 

Dodatkowo, ponieważ ten dataset jest mocno niezbalansowany, użyjemy specjalnego samplera do losowania przykładów treningowych, tak aby do uczenia sieć widziała tyle samo przykładów z każdej klasy.

Ponieważ nazwiska mogą mieć różne długości będziemy rozważać `batch_size = 1` w tym notebooku (choć implementacje modeli powinny działać dla dowolnych wartości `batch_size`!)

In [ ]:
!wget https://download.pytorch.org/tutorial/data.zip
!unzip data.zip

--2021-01-18 19:05:37--  https://download.pytorch.org/tutorial/data.zip
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving download.pytorch.org (download.pytorch.org)... 65.9.58.14, 65.9.58.98, 65.9.58.110, ...
Connecting to download.pytorch.org (download.pytorch.org)|65.9.58.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2882130 (2,7M) [application/zip]
Saving to: ‘data.zip.3’

data.zip.3          100%[===================>]   2,75M  1,87MB/s    in 1,5s    

2021-01-18 19:05:39 (1,87 MB/s) - ‘data.zip.3’ saved [2882130/2882130]

Archive:  data.zip
replace data/eng-fra.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [2]:
# NOTE: you can change the seed or remove it completely if you like
torch.manual_seed(1337)

data_dir = 'data/names'

data = []
targets = [] 
label_to_idx = {}

# read each natonality file and process data 
for label, file_name in enumerate(os.listdir(data_dir)):
    
    label_to_idx[label] = file_name.split('.')[0].lower()
    
    names = read_lines(os.path.join(data_dir, file_name))
    data += [line_to_tensor(name) for name in names]
    targets += len(names) * [label]

# split into train and test indices
test_frac = 0.1
n_test = int(test_frac * len(targets))
test_ind = np.random.choice(len(targets), size=n_test, replace=False)
train_ind = np.setdiff1d(np.arange(len(targets)), test_ind)

targets = torch.tensor(targets)
train_targets = targets[train_ind]

# calculate weights for BalancedSampler
uni, counts = np.unique(train_targets, return_counts=True)
weight_per_class = len(targets) / counts
weight = [weight_per_class[c] for c in train_targets]
# preapre the sampler
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights=weight, num_samples=len(weight)) 

train_dataset = ListDataset(data=[x for i, x in enumerate(data) if i in train_ind], targets=train_targets)
train_loader = DataLoader(train_dataset, shuffle=False, batch_size=1, sampler=sampler)

test_dataset = ListDataset(data=[x for i, x in enumerate(data) if i in test_ind], targets=targets[test_ind])
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=1)

In [82]:
# check out the content of the dataset
for i, (x, y) in enumerate(train_loader):
    print(x.shape)
    for letter_onehot in x[0]:
        print(all_letters[torch.argmax(letter_onehot)], end="")

print("x.shape:", x.shape)
print("name: ", end="")
for letter_onehot in x[0]:
    print(all_letters[torch.argmax(letter_onehot)], end="")

print("\ny:", label_to_idx[y.item()])

torch.Size([1, 5, 52])
Rochatorch.Size([1, 6, 52])
Kiharatorch.Size([1, 8, 52])
Mkrtumovtorch.Size([1, 7, 52])
Knightstorch.Size([1, 5, 52])
Hrulatorch.Size([1, 7, 52])
Durantetorch.Size([1, 6, 52])
Haddadtorch.Size([1, 7, 52])
Michelitorch.Size([1, 8, 52])
Maneatestorch.Size([1, 5, 52])
Pfafftorch.Size([1, 6, 52])
Ilsleytorch.Size([1, 8, 52])
Fourakistorch.Size([1, 8, 52])
Dziedzictorch.Size([1, 6, 52])
Mcleantorch.Size([1, 6, 52])
Suzukitorch.Size([1, 3, 52])
Ngotorch.Size([1, 10, 52])
Dobridnyuktorch.Size([1, 6, 52])
Giollatorch.Size([1, 7, 52])
Pollocktorch.Size([1, 3, 52])
Vuutorch.Size([1, 6, 52])
Gormantorch.Size([1, 7, 52])
Tillenstorch.Size([1, 7, 52])
Niftriktorch.Size([1, 8, 52])
Fakhourytorch.Size([1, 5, 52])
Fermitorch.Size([1, 7, 52])
Mahoneytorch.Size([1, 4, 52])
Kangtorch.Size([1, 6, 52])
Duartetorch.Size([1, 5, 52])
Rizzotorch.Size([1, 3, 52])
Wantorch.Size([1, 8, 52])
Campbelltorch.Size([1, 4, 52])
Zinotorch.Size([1, 6, 52])
Burdentorch.Size([1, 5, 52])
Changtorch.Siz

Tostorch.Size([1, 6, 52])
Daniautorch.Size([1, 6, 52])
Sataketorch.Size([1, 4, 52])
Lobotorch.Size([1, 7, 52])
Joynsontorch.Size([1, 6, 52])
Svotaktorch.Size([1, 8, 52])
Hasnulintorch.Size([1, 6, 52])
Crespotorch.Size([1, 2, 52])
Lytorch.Size([1, 4, 52])
Melotorch.Size([1, 4, 52])
Ivortorch.Size([1, 9, 52])
Fleischertorch.Size([1, 6, 52])
Janochtorch.Size([1, 3, 52])
Yootorch.Size([1, 2, 52])
Sotorch.Size([1, 6, 52])
Francotorch.Size([1, 7, 52])
Kringostorch.Size([1, 3, 52])
Shetorch.Size([1, 9, 52])
Lynsmeiertorch.Size([1, 9, 52])
Yangarbertorch.Size([1, 4, 52])
Zhuotorch.Size([1, 4, 52])
Ennstorch.Size([1, 8, 52])
Christietorch.Size([1, 3, 52])
Caotorch.Size([1, 7, 52])
Schrecktorch.Size([1, 4, 52])
Tsaitorch.Size([1, 7, 52])
Rosalestorch.Size([1, 6, 52])
Gavriltorch.Size([1, 11, 52])
Sienkiewicztorch.Size([1, 7, 52])
Arakawatorch.Size([1, 5, 52])
Sannatorch.Size([1, 8, 52])
Majewskitorch.Size([1, 9, 52])
Desantigotorch.Size([1, 8, 52])
Melsbachtorch.Size([1, 6, 52])
Masonitorch.Size

Locatellitorch.Size([1, 5, 52])
Tostotorch.Size([1, 4, 52])
Riostorch.Size([1, 4, 52])
Teohtorch.Size([1, 6, 52])
Wilmottorch.Size([1, 10, 52])
Schneijdertorch.Size([1, 7, 52])
Schultetorch.Size([1, 3, 52])
Lactorch.Size([1, 7, 52])
Onoharatorch.Size([1, 5, 52])
Flanntorch.Size([1, 6, 52])
Hedleytorch.Size([1, 6, 52])
Mohrentorch.Size([1, 6, 52])
Kotaratorch.Size([1, 8, 52])
Christoutorch.Size([1, 2, 52])
Dotorch.Size([1, 7, 52])
Laganastorch.Size([1, 4, 52])
Tumatorch.Size([1, 8, 52])
Silveiratorch.Size([1, 7, 52])
Okazakitorch.Size([1, 9, 52])
Jaskolskitorch.Size([1, 8, 52])
Provenzatorch.Size([1, 5, 52])
Ottentorch.Size([1, 11, 52])
Kouropoulostorch.Size([1, 10, 52])
Sniegowskitorch.Size([1, 8, 52])
Kanesakatorch.Size([1, 4, 52])
Yeontorch.Size([1, 4, 52])
Ryomtorch.Size([1, 6, 52])
Slazaktorch.Size([1, 4, 52])
Aquatorch.Size([1, 7, 52])
Rowlandtorch.Size([1, 6, 52])
Pavliktorch.Size([1, 8, 52])
Bustillotorch.Size([1, 6, 52])
Truongtorch.Size([1, 10, 52])
Priemykhovtorch.Size([1, 4,

Rosetorch.Size([1, 6, 52])
Guerratorch.Size([1, 4, 52])
Mengtorch.Size([1, 5, 52])
Powertorch.Size([1, 5, 52])
Rochatorch.Size([1, 9, 52])
Donnchadhtorch.Size([1, 12, 52])
Jarmuhamedovtorch.Size([1, 8, 52])
Hasimototorch.Size([1, 4, 52])
Belltorch.Size([1, 8, 52])
Seaghdhatorch.Size([1, 3, 52])
Kimtorch.Size([1, 9, 52])
Alexandartorch.Size([1, 6, 52])
Phocastorch.Size([1, 4, 52])
Thantorch.Size([1, 6, 52])
Seniortorch.Size([1, 6, 52])
Moorentorch.Size([1, 6, 52])
Sarkistorch.Size([1, 6, 52])
Tamarotorch.Size([1, 8, 52])
Piskovoytorch.Size([1, 8, 52])
Oirschottorch.Size([1, 6, 52])
Nakanotorch.Size([1, 5, 52])
Luongtorch.Size([1, 3, 52])
Siutorch.Size([1, 8, 52])
Kaminskitorch.Size([1, 4, 52])
Lobotorch.Size([1, 6, 52])
Torrestorch.Size([1, 7, 52])
Kirchmatorch.Size([1, 6, 52])
Rompuytorch.Size([1, 7, 52])
Kefalastorch.Size([1, 8, 52])
Hudyakovtorch.Size([1, 7, 52])
Fordhamtorch.Size([1, 8, 52])
Mcgregortorch.Size([1, 7, 52])
Zhminkotorch.Size([1, 4, 52])
Yangtorch.Size([1, 8, 52])
Vihl

Ciardhatorch.Size([1, 6, 52])
Grahamtorch.Size([1, 7, 52])
Osaragitorch.Size([1, 6, 52])
Grahamtorch.Size([1, 11, 52])
Walentowicztorch.Size([1, 9, 52])
Kawagishitorch.Size([1, 4, 52])
Adamtorch.Size([1, 8, 52])
Ramakerstorch.Size([1, 7, 52])
Ribeirotorch.Size([1, 4, 52])
Qiaotorch.Size([1, 4, 52])
Zhantorch.Size([1, 7, 52])
Douglastorch.Size([1, 8, 52])
Mkervalitorch.Size([1, 6, 52])
Wiatertorch.Size([1, 8, 52])
Medeirostorch.Size([1, 8, 52])
Zharkikhtorch.Size([1, 7, 52])
Pelevintorch.Size([1, 7, 52])
Gouveiatorch.Size([1, 9, 52])
Kowalczyktorch.Size([1, 9, 52])
Hayashidatorch.Size([1, 3, 52])
Yeotorch.Size([1, 5, 52])
Hadadtorch.Size([1, 8, 52])
Palmeirotorch.Size([1, 2, 52])
Yutorch.Size([1, 3, 52])
Mantorch.Size([1, 7, 52])
Svobodatorch.Size([1, 7, 52])
Serafintorch.Size([1, 5, 52])
Goochtorch.Size([1, 9, 52])
Pavlushintorch.Size([1, 8, 52])
Bursinostorch.Size([1, 4, 52])
Tejctorch.Size([1, 8, 52])
Mangnalltorch.Size([1, 7, 52])
Maessentorch.Size([1, 3, 52])
Huotorch.Size([1, 10, 

...

Giangtorch.Size([1, 4, 52])
Kaultorch.Size([1, 3, 52])
Nietorch.Size([1, 3, 52])
Phitorch.Size([1, 5, 52])
Fauretorch.Size([1, 5, 52])
Giangtorch.Size([1, 3, 52])
Chatorch.Size([1, 6, 52])
Uesugitorch.Size([1, 3, 52])
Weitorch.Size([1, 3, 52])
Ngotorch.Size([1, 6, 52])
Gormantorch.Size([1, 5, 52])
Salibtorch.Size([1, 6, 52])
Demalltorch.Size([1, 5, 52])
Nasertorch.Size([1, 4, 52])
Lingtorch.Size([1, 7, 52])
Vescovitorch.Size([1, 3, 52])
Paktorch.Size([1, 6, 52])
Smolaktorch.Size([1, 6, 52])
Bohmertorch.Size([1, 5, 52])
Boschtorch.Size([1, 7, 52])
Gwozdektorch.Size([1, 5, 52])
Mariatorch.Size([1, 8, 52])
Buggenumtorch.Size([1, 6, 52])
Guerratorch.Size([1, 4, 52])
Kwaktorch.Size([1, 6, 52])
Thorpetorch.Size([1, 4, 52])
Bachtorch.Size([1, 5, 52])
Colontorch.Size([1, 6, 52])
Mooneytorch.Size([1, 6, 52])
Yasudatorch.Size([1, 8, 52])
Faltysektorch.Size([1, 4, 52])
Doantorch.Size([1, 5, 52])
Assaftorch.Size([1, 7, 52])
Kennedytorch.Size([1, 8, 52])
Altamuratorch.Size([1, 5, 52])
Machatorch.Si

Casalestorch.Size([1, 8, 52])
Davidsontorch.Size([1, 5, 52])
Riccitorch.Size([1, 5, 52])
Fletttorch.Size([1, 4, 52])
Jiantorch.Size([1, 6, 52])
Crespotorch.Size([1, 12, 52])
Delafontainetorch.Size([1, 6, 52])
Nasatotorch.Size([1, 5, 52])
Riccitorch.Size([1, 3, 52])
Paktorch.Size([1, 5, 52])
Giangtorch.Size([1, 5, 52])
Nursetorch.Size([1, 7, 52])
Bulgaritorch.Size([1, 4, 52])
Kieutorch.Size([1, 8, 52])
Fergusontorch.Size([1, 4, 52])
Ngaitorch.Size([1, 4, 52])
Wangtorch.Size([1, 9, 52])
Mcdougalltorch.Size([1, 7, 52])
Traverttorch.Size([1, 9, 52])
Gutierreztorch.Size([1, 8, 52])
Pinheirotorch.Size([1, 4, 52])
Seiftorch.Size([1, 6, 52])
Thomastorch.Size([1, 6, 52])
Garciatorch.Size([1, 6, 52])
Duboistorch.Size([1, 8, 52])
Palmeirotorch.Size([1, 4, 52])
Belotorch.Size([1, 7, 52])
Jahnyuktorch.Size([1, 5, 52])
Katsutorch.Size([1, 5, 52])
Starktorch.Size([1, 5, 52])
Quachtorch.Size([1, 8, 52])
Pinheirotorch.Size([1, 7, 52])
Pfeifertorch.Size([1, 6, 52])
Evasontorch.Size([1, 6, 52])
Godwintor

Roosatorch.Size([1, 8, 52])
Alamillatorch.Size([1, 8, 52])
Ichiharatorch.Size([1, 3, 52])
Tontorch.Size([1, 9, 52])
Lesauvagetorch.Size([1, 5, 52])
Aodhatorch.Size([1, 4, 52])
Kooltorch.Size([1, 12, 52])
Antimisiaristorch.Size([1, 4, 52])
Thantorch.Size([1, 4, 52])
Bachtorch.Size([1, 6, 52])
Almasitorch.Size([1, 6, 52])
Dunloptorch.Size([1, 10, 52])
Scheinbergtorch.Size([1, 6, 52])
Rzehaktorch.Size([1, 5, 52])
Saulttorch.Size([1, 6, 52])
Thomastorch.Size([1, 4, 52])
Raistorch.Size([1, 8, 52])
Reinderstorch.Size([1, 6, 52])
Ibaneztorch.Size([1, 6, 52])
Letsostorch.Size([1, 5, 52])
Kokantorch.Size([1, 5, 52])
Romaotorch.Size([1, 4, 52])
Pagetorch.Size([1, 4, 52])
Kozatorch.Size([1, 8, 52])
Ferreirotorch.Size([1, 12, 52])
Beklemischevtorch.Size([1, 7, 52])
Narvaeztorch.Size([1, 6, 52])
Millartorch.Size([1, 5, 52])
Tykaltorch.Size([1, 9, 52])
Ricchettitorch.Size([1, 6, 52])
Antountorch.Size([1, 5, 52])
Duongtorch.Size([1, 10, 52])
Miksatkovatorch.Size([1, 7, 52])
Gouveiatorch.Size([1, 6, 5

Freitorch.Size([1, 4, 52])
Liaotorch.Size([1, 6, 52])
Morcostorch.Size([1, 6, 52])
Campostorch.Size([1, 9, 52])
Cheryshevtorch.Size([1, 5, 52])
Assaftorch.Size([1, 8, 52])
Stamatastorch.Size([1, 7, 52])
Borovkatorch.Size([1, 3, 52])
Youtorch.Size([1, 7, 52])
Oquendotorch.Size([1, 7, 52])
Koemanstorch.Size([1, 10, 52])
Cunninghamtorch.Size([1, 5, 52])
Fletttorch.Size([1, 7, 52])
Zhelaevtorch.Size([1, 4, 52])
Abditorch.Size([1, 3, 52])
Kautorch.Size([1, 5, 52])
Totahtorch.Size([1, 14, 52])
Giannakopoulostorch.Size([1, 9, 52])
Markwardttorch.Size([1, 6, 52])
Molloytorch.Size([1, 8, 52])
Tresedertorch.Size([1, 9, 52])
Niftericktorch.Size([1, 9, 52])
Jatsevichtorch.Size([1, 7, 52])
Patricktorch.Size([1, 15, 52])
Panayiotopoulostorch.Size([1, 4, 52])
Reidtorch.Size([1, 7, 52])
Cassidytorch.Size([1, 6, 52])
Gordontorch.Size([1, 9, 52])
Sheinfeldtorch.Size([1, 3, 52])
Portorch.Size([1, 9, 52])
Wondracektorch.Size([1, 4, 52])
Rhystorch.Size([1, 7, 52])
Dyersontorch.Size([1, 4, 52])
Rangtorch.Si

Chotorch.Size([1, 5, 52])
Caomhtorch.Size([1, 7, 52])
Osagawatorch.Size([1, 4, 52])
Kiratorch.Size([1, 7, 52])
Althuistorch.Size([1, 6, 52])
Vargastorch.Size([1, 9, 52])
Strilakostorch.Size([1, 3, 52])
Fuxtorch.Size([1, 9, 52])
Gagliarditorch.Size([1, 7, 52])
Laurenttorch.Size([1, 8, 52])
Flanagantorch.Size([1, 4, 52])
Chautorch.Size([1, 6, 52])
Hargertorch.Size([1, 5, 52])
Zitkatorch.Size([1, 5, 52])
Amaritorch.Size([1, 8, 52])
Kurofujitorch.Size([1, 9, 52])
Hendersontorch.Size([1, 8, 52])
Muyskenstorch.Size([1, 6, 52])
Wyrzyktorch.Size([1, 6, 52])
Garbertorch.Size([1, 6, 52])
Malouftorch.Size([1, 10, 52])
Vassilikostorch.Size([1, 5, 52])
Chieutorch.Size([1, 9, 52])
Katschkertorch.Size([1, 8, 52])
Marszaektorch.Size([1, 12, 52])
Konstantatostorch.Size([1, 5, 52])
Spinitorch.Size([1, 8, 52])
Brucknertorch.Size([1, 9, 52])
Bagmevskitorch.Size([1, 9, 52])
Rovigattitorch.Size([1, 5, 52])
Nacartorch.Size([1, 7, 52])
Faerbertorch.Size([1, 4, 52])
Veentorch.Size([1, 5, 52])
Kuntztorch.Size([

## Zadanie 1. (2 pkt.)

Zaimplementuj "zwykłą" sieć rekurencyjną. 
![rnn](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png)

* W klasie `RNN` należy zainicjalizować potrzebne wagi oraz zaimplementować główną logikę dla pojedynczej chwili czasowej $x_t$
* Wyjście z sieci możemy mieć dowolny rozmiar, potrzebna jest również warstwa przekształacjąca stan ukryty na wyjście.
* W pętli uczenia należy dodać odpowiednie wywołanie sieci. HINT: pamiętać o iterowaniu po wymiarze "czasowym".
* Zalecane jest użycie aktywacji na warstwie liczącej reprezentacje `hidden` tak, aby nie "eksplodowała", np. `tanh`.


In [414]:
class RNN(torch.nn.Module):
    
    def __init__(self, 
                 input_size: int,
                 hidden_size: int, 
                 output_size: int):
        """
        :param input_size: int
            Dimensionality of the input vector
        :param hidden_size: int
            Dimensionality of the hidden space
        :param output_size: int
            Desired dimensionality of the output vector
        """
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.input_to_hidden = torch.nn.Linear(input_size + hidden_size, self.hidden_size)
        
        self.hidden_to_output = torch.nn.Linear(input_size + hidden_size, output_size)
    
    # for the sake of simplicity a single forward will process only a single timestamp 
    def forward(self, 
                input: torch.tensor, 
                hidden: torch.tensor) -> Tuple[torch.tensor, torch.tensor]:
        """
        :param input: torch.tensor 
            Input tesnor for a single observation at timestep t
            shape [batch_size, input_size]
        :param hidden: torch.tensor
            Representation of the memory of the RNN from previous timestep
            shape [batch_size, hidden_size]
        """

        combined = torch.cat([input, hidden], dim=1) 
        hidden = self.input_to_hidden(combined)
        output =  torch.tanh(self.hidden_to_output(combined))
        return output, hidden
    
    def init_hidden(self, batch_size: int) -> torch.Tensor:
        """
        Returns initial value for the hidden state
        """
        return torch.zeros(batch_size, self.hidden_size, requires_grad=True)

### Pętla uczenia

## n_class = len(label_to_idx)


In [415]:
n_class = len(label_to_idx)

# initialize network and optimizer
rnn = RNN(n_letters, 256, n_class)
optimizer = torch.optim.SGD(rnn.parameters(), lr=0.005)   

# we will train for only a single epoch 
epochs = 1


# main loop
for epoch in range(epochs):
    
    loss_buffer = []
    for i, (x, y) in enumerate(train_loader):  
        optimizer.zero_grad()
        # get initial hidden state
        hidden = rnn.init_hidden(x.shape[0])
        # get output for the sample, remember that we treat it as a sequence
        # so you need to iterate over the 2nd, time dimensiotn
        seq_len = x.shape[1]
        output = None
        for current_iteration in range(seq_len):
            output, hidden= rnn.forward(x[:,current_iteration],hidden)
        loss = cross_entropy(output, y)
        loss.backward()
        optimizer.step()  
        
        loss_buffer.append(loss.item())
        
        if i % 1000 == 1:
            print(f"Epoch: {epoch} Progress: {100 * i/len(train_loader):2.0f}% Loss: {np.mean(loss_buffer):.3f}")
            loss_buffer = []
    

# evaluate on the test set
with torch.no_grad():
    ps = []
    ys = []
    correct = 0
    for i, (x, y) in enumerate(test_loader):
        ys.append(y.numpy())

        hidden = rnn.init_hidden(x.shape[0])
        seq_len = x.shape[1]
 
        for current_iteration in range(seq_len):
            output, hidden= rnn.forward(x[:,current_iteration],hidden)

        pred = output.argmax(dim=1)
        ps.append(pred.cpu().numpy())
    
    ps = np.concatenate(ps, axis=0)
    ys = np.concatenate(ys, axis=0)
    f1 = f1_score(ys, ps, average='weighted')
    
    print(f"Final F1 score: {f1:.2f}")
    assert f1 > 0.15, "You should get over 0.15 f1 score, try changing some hiperparams!"

Epoch: 0 Progress:  0% Loss: 2.896
Epoch: 0 Progress:  6% Loss: 2.872
Epoch: 0 Progress: 11% Loss: 2.817
Epoch: 0 Progress: 17% Loss: 2.752
Epoch: 0 Progress: 22% Loss: 2.672
Epoch: 0 Progress: 28% Loss: 2.590
Epoch: 0 Progress: 33% Loss: 2.520
Epoch: 0 Progress: 39% Loss: 2.490
Epoch: 0 Progress: 44% Loss: 2.477
Epoch: 0 Progress: 50% Loss: 2.399
Epoch: 0 Progress: 55% Loss: 2.395
Epoch: 0 Progress: 61% Loss: 2.387
Epoch: 0 Progress: 66% Loss: 2.328
Epoch: 0 Progress: 72% Loss: 2.308
Epoch: 0 Progress: 77% Loss: 2.290
Epoch: 0 Progress: 83% Loss: 2.253
Epoch: 0 Progress: 89% Loss: 2.243
Epoch: 0 Progress: 94% Loss: 2.226
Epoch: 0 Progress: 100% Loss: 2.234
Final F1 score: 0.16


## Zadanie 2. (0.5 pkt.)
Zaimplementuj funkcje `predict`, która przyjmuje nazwisko w postaci stringa oraz model RNN i wypisuje 3 najlepsze predykcje narodowości dla tego nazwiska razem z ich logitami.

**Hint**: Przyda się tutaj jedna z funkcji z pierwszej komórki notebooka.

## Zadanie 2. (0.5 pkt.)
Zaimplementuj funkcje `predict`, która przyjmuje nazwisko w postaci stringa oraz model RNN i wypisuje 3 najlepsze predykcje narodowości dla tego nazwiska razem z ich logitami.

**Hint**: Przyda się tutaj jedna z funkcji z pierwszej komórki notebooka.

In [416]:
def predict(name: str, rnn: RNN):
    coded_name = line_to_tensor(name).unsqueeze(0)
    hidden = rnn.init_hidden(1)
    for current_iteration in range(len(name)):
            output, hidden= rnn.forward(torch.tensor(coded_name[:,current_iteration]),hidden)
    """Prints the name and model's top 3 predictions with scores"""
    values , indices = torch.sort(output,descending =True)
    for i in range(3):
        print( str(i+1) +": " + label_to_idx[int(indices[0][i])] +" " +str(float(values[0][i])))
    return output

In [417]:
some_names = ["Satoshi", "Jackson", "Schmidhuber", "Hinton", "Kowalski"]

for name in some_names:
    print(name)
    predict(name, rnn)

Satoshi
1: polish 0.7553368806838989
2: japanese 0.7113988995552063
3: arabic 0.4912700653076172
Jackson
1: scottish 0.44876450300216675
2: russian 0.4286384880542755
3: english 0.09032492339611053
Schmidhuber
1: german 0.6577728390693665
2: dutch 0.30118197202682495
3: czech -0.48624691367149353
Hinton
1: scottish 0.6112527847290039
2: russian 0.22445842623710632
3: english 0.2193748503923416
Kowalski
1: polish 0.8502445816993713
2: japanese 0.48734617233276367
3: russian 0.2986702024936676


<ipython-input-416-bc9a69c01060>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output, hidden= rnn.forward(torch.tensor(coded_name[:,current_iteration]),hidden)


## Zadanie 3 (4 pkt.)
Ostatnim zadaniem jest implementacji komórki i sieci LSTM. 

![lstm](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)

* W klasie `LSTMCell` ma znaleźć się główna loginka LSTMa, czyli wszystkie wagi do stanów `hidden` i `cell` jak i bramek kontrolujących te stany. 
* W klasie `LSTM` powinno znaleźć się wywołanie komórki LSTM, HINT: poprzednio było w pętli uczenia, teraz przenisiemy to do klasy modelu.
* W pętli uczenia należy uzupełnić brakujące wywołania do uczenia i ewaluacji modelu.

Zdecydowanie polecam [materiały Chrisa Olaha](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) do zarówno zrozumienia jak i ściągi do wzorów.

Zadaniem jest osiągnięcie wartości `f1_score` lepszej niż na sieci RNN, przy prawidłowej implementacji nie powinno być z tym problemów używając podanych hiperparametrów. Dozwolona jest oczywiście zmiana `random seed`.

#### Komórka LSTM

In [371]:
class LSTMCell(torch.nn.Module):

    def __init__(self, 
                 input_size: int, 
                 hidden_size: int):
        """
        :param input_size: int
            Dimensionality of the input vector
        :param hidden_size: int
            Dimensionality of the hidden space
        """
        
        super(LSTMCell, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        # initialize LSTM weights 
        # NOTE: there are different approaches that are all correct 
        # (e.g. single matrix for all input opperations), you can pick
        # whichever you like for this task
        self.Wii = torch.nn.Linear(input_size,self.hidden_size)
        self.Whi = torch.nn.Linear(self.hidden_size,self.hidden_size)
        self.Wif = torch.nn.Linear(input_size,self.hidden_size)
        self.Whf = torch.nn.Linear(self.hidden_size,self.hidden_size)
        self.Wig = torch.nn.Linear(input_size,self.hidden_size)
        self.Whg = torch.nn.Linear(self.hidden_size,self.hidden_size)
        self.Wio = torch.nn.Linear(input_size,self.hidden_size)
        self.Who = torch.nn.Linear(self.hidden_size,self.hidden_size)

    def forward(self, 
                input: torch.tensor, 
                states: Tuple[torch.tensor, torch.tensor]) -> Tuple[torch.tensor, torch.tensor]:
        
        hidden, cell = states
        # Compute input, forget, and output gates
        # then compute new cell state and hidden state
        # see http://colah.github.io/posts/2015-08-Understanding-LSTMs/ print("no jestem")
        i_t = torch.sigmoid(self.Wii(input) + self.Whi(hidden))
        f_t = torch.sigmoid(self.Wif(input) + self.Whf(hidden))
        g_t = torch.tanh(self.Wig(input) + self.Whg(hidden))
        o_t = torch.sigmoid(self.Wio(input) + self.Who(hidden))
        cell = f_t * cell + i_t * g_t
        hidden = o_t * torch.tanh(cell)
    
    
        
        return hidden, cell

### Klasa modelu LSTM

In [374]:
class LSTM(torch.nn.Module):

    def __init__(self, 
                 input_size: int, 
                 hidden_size: int):
        """
        :param input_size: int
            Dimensionality of the input vector
        :param hidden_size: int
            Dimensionality of the hidden space
        """
        
        super(LSTM, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        
    
        self.cell = LSTMCell(input_size=input_size, hidden_size=hidden_size)
        
    def forward(self, 
                input: torch.tensor) -> Tuple[torch.tensor, torch.tensor]:
        """
        :param input: torch.tensor 
            Input tesnor for a single observation at timestep t
            shape [batch_size, input_size]
        Returns Tuple of two torch.tensors, both of shape [seq_len, batch_size, hidden_size]
        
        """
    
        batch_size = input.shape[0]
        
        hidden, cell = self.init_hidden_cell(batch_size)
        hiddens = []
        cells = []
        
        # this time we will process the whole sequence in the forward method
        # as oppose to the previous exercise, remember to loop over the timesteps
        
        time_steps = input.shape[1]
        for i in range(time_steps):  
            hidden, cell = self.cell.forward(input[:,i],(hidden,cell))
            hiddens.append(hidden)
            cells.append(cell)
        hiddens = torch.stack(hiddens)
        cells = torch.stack(cells)
        return hiddens, cells
    
    def init_hidden_cell(self, batch_size):
        """
        Returns initial value for the hidden and cell states
        """
        return (torch.zeros(batch_size, self.hidden_size, requires_grad=True), 
                torch.zeros(batch_size, self.hidden_size, requires_grad=True))

### Pętla uczenia

In [399]:
from itertools import chain

# torch.manual_seed(1337)

# build data loaders
train_loader = DataLoader(train_dataset, batch_size=1, sampler=sampler)
test_loader = DataLoader(test_dataset, batch_size=1)

# initialize the lstm with an additional cliassifier layer at the top
lstm = LSTM(input_size=len(all_letters), hidden_size=128)
clf = torch.nn.Linear(in_features=128, out_features=len(label_to_idx))

# initialize a optimizer
params = chain(lstm.parameters(), clf.parameters())
optimizer = torch.optim.Adam(params, lr=0.005) 

# we will train for only a single epoch 
epoch = 1

# main loop
for epoch in range(epoch):
    
    loss_buffer = []
    
    for i, (x, y) in enumerate(train_loader):   
        optimizer.zero_grad()
        
        # get output for the sample, remember that we treat it as a sequence
        # so you need to iterate over the sequence length here
        # don't forget about the classifier! 
        
           
        hiddens, cells= lstm.forward(x)
        output = clf(hiddens[-1][-1].unsqueeze(0))
        
        # calucate the loss
        loss = cross_entropy(output, y)
        loss.backward()
        optimizer.step()                                
        
        loss_buffer.append(loss.item())
        
        if i % 1000 == 1:
            print(f"Epoch: {epoch} Progress: {100 * i/len(train_loader):2.0f}% Loss: {np.mean(loss_buffer):.3f}")
            loss_buffer = []

# evaluate on the test set
with torch.no_grad():
    
    ps = []
    ys = []
    for i, (x, y) in enumerate(test_loader): 
        
        ys.append(y.numpy())
        
        hiddens, cells= lstm.forward(x)
        output = clf(hiddens[-1][-1].unsqueeze(0))

        pred = output.argmax(dim=1)
        ps.append(pred.cpu().numpy())
    
    ps = np.concatenate(ps, axis=0)
    ys = np.concatenate(ys, axis=0)
    f1 = f1_score(ys, ps, average='weighted')
    
    print(f"Final F1 score: {f1:.2f}")
    assert f1 > 0.18, "You should get over 0.18 f1 score, try changing some hiperparams!"

Epoch: 0 Progress:  0% Loss: 2.822
Epoch: 0 Progress:  6% Loss: 2.463
Epoch: 0 Progress: 11% Loss: 1.905
Epoch: 0 Progress: 17% Loss: 1.674
Epoch: 0 Progress: 22% Loss: 1.508
Epoch: 0 Progress: 28% Loss: 1.360
Epoch: 0 Progress: 33% Loss: 1.314
Epoch: 0 Progress: 39% Loss: 1.234
Epoch: 0 Progress: 44% Loss: 1.262
Epoch: 0 Progress: 50% Loss: 1.087
Epoch: 0 Progress: 55% Loss: 1.039
Epoch: 0 Progress: 61% Loss: 0.976
Epoch: 0 Progress: 66% Loss: 0.952
Epoch: 0 Progress: 72% Loss: 0.921
Epoch: 0 Progress: 77% Loss: 0.891
Epoch: 0 Progress: 83% Loss: 0.882
Epoch: 0 Progress: 89% Loss: 0.781
Epoch: 0 Progress: 94% Loss: 0.841
Epoch: 0 Progress: 100% Loss: 0.810
Final F1 score: 0.23


## Zadanie 4. (0.5 pkt.)
Zaimplementuj analogiczną do funkcji `predict` z zadania 2 dla modelu `lstm+clf`.


In [403]:
def predict_lstm(name: str, lstm: LSTM, clf: torch.nn.Module):
    coded_name = line_to_tensor(name).unsqueeze(0)
    hiddens, cells= lstm.forward(coded_name)
    output = clf(hiddens[-1][-1].unsqueeze(0))
    """Prints the name and model's top 3 predictions with scores"""
    values1 , indices1 = torch.sort(output,descending =True)
    for i in range(3):
        print( str(i+1) +": " + label_to_idx[int(indices1[0][i])] +" " +str(float(values1[0][i])))
    return output

In [404]:
# test your lstm predictor
some_names = ["Satoshi", "Jackson", "Schmidhuber", "Hinton", "Kowalski"]
    
for name in some_names:
    print(name)
    predict_lstm(name, lstm, clf)

Satoshi
1: japanese 6.050233840942383
2: arabic 4.847599983215332
3: italian 4.128917217254639
Jackson
1: scottish 4.180373191833496
2: english 3.66139554977417
3: irish 1.0572185516357422
Schmidhuber
1: german 4.974162578582764
2: czech 2.2213540077209473
3: english 2.001784324645996
Hinton
1: english 3.3148698806762695
2: dutch 2.3090853691101074
3: german 1.9950289726257324
Kowalski
1: polish 10.050779342651367
2: russian 2.6385436058044434
3: czech 2.4632649421691895
